In [133]:
import dash
from dash import html, dcc, Dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from sklearn import linear_model

In [134]:
hours_per_project = pd.read_excel('hours_per_project_3.xlsx')

In [135]:
hours_per_customer = pd.read_csv('hours_per_customer.csv')

In [136]:
ser_ins_war_hours = pd.read_csv('ser_ins_war_hours.csv')

In [137]:
# step 1: map chart for service hours per Year and Country/region
# List of years to loop through
map_hours = hours_per_project.groupby('Country/region').agg({'Hours':'sum'}).reset_index()

custom_colorscale = 'YlGnBu'

# Create choropleth map trace
choropleth_map = go.Choropleth(
    locations=map_hours['Country/region'],
    z=map_hours['Hours'],
    text=map_hours['Country/region'],
    colorscale=custom_colorscale,  # Set custom colorscale
    colorbar={'title': 'Total Hours by Country'}
)

layout = dict(geo={'showframe': False, 'projection': {'type':'mercator'}},
)

fig = go.Figure(data=[choropleth_map], layout=layout)
fig.show()

In [138]:
# step 2: warranty, service, installation hours (stacked bar chart)
total_swi_hours = ser_ins_war_hours.groupby([ 'Year', 'Project group']).agg({'Hours':'sum'}).reset_index()

fig1c = px.bar(total_swi_hours, 
               x='Hours', 
               y='Project group',
               title='Proportion of warranty, service and installation hours (2018 - 2023)',
               color = 'Year', 
               barmode = 'group', 
               orientation='h',  
)

fig1c.update_traces(textposition='outside')

fig1c.show()

In [139]:
# travel and work hours repartition
category_hours = hours_per_project.groupby(['Project group', 'Travel or Work']).agg({'Hours':'sum'}).reset_index()

fig2 = px.bar(category_hours.sort_values(by='Hours'), 
              x='Hours', 
              y='Project group', 
              title='Travel and work hours in aftermarket service (2018 - 2023)',
              color = 'Travel or Work', 
              barmode = 'group', 
              orientation='h', 
              text='Hours' 
)

fig2.update_traces(textposition='outside')

In [140]:
fig4 = px.pie(hours_per_project.groupby('Travel or Work').agg({'Hours':'sum'}).reset_index(), values="Hours", names="Travel or Work", title='Work and Travel Hours (2018 - 2023)', color_discrete_sequence=px.colors.qualitative.Plotly)

# showing the plot
fig4.show()

In [141]:
workers_hours = hours_per_project.groupby(['Worker', 'Year']).agg({'Hours':'sum'}).reset_index()

In [142]:
workers_hours['Utilization'] = (workers_hours['Hours']/1756)*100

In [143]:
color_map = {'technician 1': '#1f77b4', 
             'technician 2': '#ff7f0e',
             'technician 3': '#2ca02c',
             'technician 4': '#d62728', 
             'technician 5': '#9467bd',
             'technician 6': '#8c564b', 
             'technician 7': '#e377c2', 
             'technician 8': '#7f7f7f', 
             'technician 9': '#bcbd22', 
             'technician 10': '#17becf', 
             'technician 11': '#ff5733', 
             'technician 12': '#4A90E2'
}

fig5 = px.line(workers_hours, x='Year', y='Utilization', color='Worker', markers=True,
               labels={'Utilization':'Utilization (%)', 'Year':'Year'},
               title='Technicians utilization (2018 - 2023)',
               color_discrete_map=color_map)

fig5.add_shape(type='line', x0=workers_hours['Year'].min(), x1=workers_hours['Year'].max(), y0=100, y1=100,
               line=dict(color='red', dash='dash'))

fig5.show()

In [144]:
hours_corr = hours_per_customer.select_dtypes(include=['float64']).corr()
hours_corr = hours_corr.applymap(lambda x: round(x, 3))

In [145]:
fig7 = px.imshow(hours_corr, text_auto=True, aspect='auto',
                x=['SER', 'WAR', 'INS', 'MAIN', 'HD', 'TRAIN'],
                y=['SER', 'WAR', 'INS', 'MAIN', 'HD', 'TRAIN']
               )
fig7.update_xaxes(side="top")
fig7.show()

In [154]:
hours_per_customer_reg = hours_per_customer[['Installation hours', 'Warranty hours']]

In [155]:
model = linear_model.LinearRegression()

In [156]:
X1 = hours_per_customer_reg.drop('Warranty hours', axis=1)
y1 = hours_per_customer_reg['Warranty hours']

X1.head()

,Installation hours
0,75.0
1,0.0
2,0.0
3,0.0
4,400.0


In [157]:
result1 = model.fit(X1, y1)
print('Intercept:', result1.intercept_)
print('Coefficient:', result1.coef_)

Intercept: 2.0659174574775605
Coefficient: [0.78356203]


In [158]:

#regression line equation
regression_line = result1.intercept_ + hours_per_customer_reg['Installation hours'] * result1.coef_


scatter = go.Scatter(
    x=hours_per_customer_reg['Installation hours'],
    y=hours_per_customer_reg['Warranty hours'],
    mode='markers',
    marker=dict(color='blue', opacity=0.45),
    name='Data Points'
)


line = go.Scatter(
    x=hours_per_customer_reg['Installation hours'],
    y=regression_line,
    mode='lines',
    line=dict(color='orange'),
    name='Regression Line'
)


layout = go.Layout(
    title='Linear regression model for warranty hours',
    xaxis=dict(title='Installation Hours'),
    yaxis=dict(title='Warranty Hours')
)

fig8 = go.Figure(data=[scatter, line], layout=layout)
fig8.show()

In [159]:
X = hours_per_customer.drop(columns=['Warranty hours', 'Customer account'], axis=1)
y = hours_per_customer['Warranty hours']

In [160]:
result = model.fit(X, y)
print('Intercept:', result.intercept_)
print('Coefficient:', result.coef_)

Intercept: -10.21744443913094
Coefficient: [ 0.19975677  0.72087452 -0.04722416 -1.90736646  0.87962653]


In [161]:
intercept = -9.776215379854346
coefficients = [0.18694672, 0.7671693, -0.04911211, -2.15437583, 1.28491407]

# Calculate predicted y values
predicted_y = intercept + X.dot(coefficients)

# Create a scatter plot
scatter = go.Scatter(
    x=y,
    y=predicted_y,
    mode='markers',
    marker=dict(color='blue', opacity=0.45),
    name='Actual vs Predicted'
)

# Create a diagonal line for perfect prediction
diagonal_line = go.Scatter(
    x=[min(y), max(y)],
    y=[min(y), max(y)],
    mode='lines',
    line=dict(color='orange', dash='dash'),
    name='Prediction line'
)

# Create layout
layout = go.Layout(
    title='Actual vs predicted warranty hours',
    xaxis=dict(title='Real Warranty hours)'),
    yaxis=dict(title='Predicted Warranty hours')
)

fig9 = go.Figure(data=[scatter, diagonal_line], layout=layout)

fig9.show()

In [179]:
# 4th Step: initialize app
app = Dash(__name__)

In [180]:
app.layout = html.Div(children=[
    html.H1('Aftermarket Service Dashboard', style={'textAlign':'center', 'backgroundColor': 'lightgrey'}),
    html.Br(), # br is linebreak
    dcc.Graph(id='choropleth_map', figure=fig),
    dcc.Graph(id='bar_chart1', figure=fig1c),
    
    html.Div([ # first dropdown square
        html.Label(['Choose a chart:'], style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': 'Work and Travel hours', 'value': 'graph1'},
                {'label': 'Work and Travel hours by Project group', 'value': 'graph2'},
            ],
            value='graph1',
            style={"width": "60%"}
        ),
        dcc.Graph(id='graph1'),
    ]),
    
    dcc.Graph(id='line_chart', figure=fig5),
    dcc.Graph(id='heatmap', figure=fig7),
    
    html.Div([ # second dropdown square
        html.Label(['Choose a chart:'], style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown2',
            options=[
                {'label': 'Linear regression model for warranty hours', 'value': 'graph3'},
                {'label': 'Actual vs predicted warranty hours with multiple features regression model', 'value': 'graph4'},
            ],
            value='graph3',
            style={"width": "60%"}
        ),
        dcc.Graph(id='graph2'),
    ]),
])


In [181]:
#6th Step: set selectable:
@app.callback(
    Output('graph1', 'figure'),
    [Input(component_id='dropdown', component_property='value')] # I am choosing from to define an output or an input - I am taking a value from a componenent that has id dropdown 
)
def select_graph(value):
    if value == 'graph2':
        return fig4
    else:
        return fig2

In [182]:
@app.callback(
    Output('graph2', 'figure'),
    [Input(component_id='dropdown2', component_property='value')]
)
def select_graph2(value):
    if value == 'graph3':
        return fig8
    else:
        return fig9

In [183]:
if __name__ == '__main__':
    app.run_server(debug=True)